<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'TFrame', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-07 11:18:38
-------------------
qualified stocks: 97
with latest results: 90
still star stocks: 48
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  2.11 L
Current:  1.21 C
-------------------
Today PnL: 42.27 K (0.35%)
Current PnL: -21.87 L (-15.51%)
CY Booked + Current PnL: -19.86 L (-14.08%)
-------------------
Total profit:  3.40 L
Total loss:  -25.27 L
-------------------
Total Booked + Current PnL: 7.12 L (6.25%)
Total Booked PnL: 28.99 L (25.42%)
Curr Year Booked PnL: 2.01 L (1.67%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.03 C
Est FTT PnL: 82.38 L (68.16%)
-------------------
Est LTT:  2.64 C
Est LTT PnL: 1.43 C (118.62%)
Deployed:  1.14 C
Current:  1.21 C
CAGR/XIRR %: 4.21%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'XIRR', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
34    HDFCBANK  1561.04  2170.00     3.67       X-LC  72.16  102433.0   
73     SBILIFE  1496.49  1924.99    -1.70       H-LC  89.69  146126.0   
66    RELIANCE  1288.53  1526.00     4.05       X-LC  95.88  223602.0   
60  PIDILITIND  2504.06  3576.00   -14.24       X-LC  40.21   89520.0   
35    HDFCLIFE   562.05   897.00    -4.90       H-LC  68.04   90669.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  XIRR  \
34      19698.0  12579.0        0.05         23.81  12.28  39.01   4.0   
73      20421.0  15577.0        0.79         16.25  10.66  28.63   NaN   
66      20014.0  17508.0       -0.40          9.83   7.83  18.43  67.0   
60      14398.0  17761.0       -0.69         19.17  19.84  42.81  10.0   
35      20413.0  21452.0        0.38         29.05  23.66  59.59  16.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
34     1.57        0.86    35.34      X40      BTT       BANKS  
73     1.31        1.23    30.18       AR      ATH   INSURANCE  
66     1.14        1.88    21.80     XY25      NTT  REFINERIES  
60     0.81        0.75    12.27      X40      BTT   CHEMICALS  
35     0.95        0.76    36.18      X40      BTT   INSURANCE

In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction     RSP   Current  \
84  TATAMOTORS   798.82  1065.0   -55.10       M-LC   83.51  138262.0   
93      VIPIND   488.80   718.0  -749.72       H-SC   93.81   61883.0   
95   WHIRLPOOL  1112.81  2270.0   -37.44       M-SC   92.78  111780.0   
1     AARTIIND   487.04   919.0    18.42       M-SC  100.00   74933.0   
6     ANGELONE  2328.67  3033.0    -6.41       H-SC   75.26  153101.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  XIRR  \
84     -25496.0  80068.0        4.05        -15.57   57.91   33.32 -49.0   
93     -33433.0  78127.0        4.03        -35.08  126.25   46.89 -47.0   
95      11627.0  92520.0        3.80         11.61   82.77  103.99  34.0   
1       -6403.0  78537.0        3.08         -7.87  104.81   88.69  -8.0   
6        1737.0  44047.0        2.44          1.15   28.77   30.25  77.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
84    -0.32        1.16    16.33     XY24      NTT       AUTO  
93    -0.43        0.52    22.06     X40N      NTT       MISC  
95     0.13        0.94    34.75      X40      NTT   DURABLES  
1     -0.08        0.63    26.36       XR      NTT  CHEMICALS  
6      0.04        1.29    21.18     X40N      NTT    FINANCE

In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost    FTT  Dev%_PE Conviction    RSP   Current  \
20   COFFEEDAY    59.14   80.0   -59.76       L-SC  62.89   59021.0   
74        SBIN   760.30  863.0   -18.54       M-LC  37.63  205136.0   
30  GREENPANEL   375.16  537.0   151.29       M-SC  43.30  112138.0   
62       QUESS   711.66  986.0      NaN        NaN   1.03   73923.0   
23         DEN    52.79   75.0    78.76       L-SC  48.45   50016.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  XIRR  \
20     -54528.0   94581.0       -2.01        -48.02  160.25  35.27   NaN   
74       3656.0   23550.0       -2.01          1.81   11.48  13.51  93.0   
30     -70940.0  149917.0       -1.73        -38.75  133.69  43.14 -17.0   
62     -81931.0  142006.0       -1.62        -52.57  192.10  38.55 -33.0   
23     -34448.0   69982.0       -1.36        -40.78  139.92  42.07 -21.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy       Category  
20    -0.58        0.50    44.45       XR      NTT         HOTELS  
74     0.16        1.73    12.38     XY25      NTT          BANKS  
30    -0.47        0.94     5.48     XY24      NTT           MISC  
62    -0.58        0.62    13.20     XY24      NTT           MISC  
23    -0.49        0.42     6.22       AR      NTT  ENTERTAINMENT

In [12]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
57        NHPC    82.08   115.00   194.74       M-MC  18.56  142766.0   
61     POLYCAB  5005.92  7541.35    18.73       H-MC  97.94  168700.0   
4   ADANIPORTS  1103.69  1583.00     2.87       M-LC  91.75  181772.0   
28       GICRE   340.22   583.00     2.32       H-MC  59.79   97227.0   
6     ANGELONE  2328.67  3033.00    -6.41       H-SC  75.26  153101.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%   XIRR  \
57       1753.0  54808.0        1.70          1.24  38.39  40.11  -54.0   
61      28534.0  42462.0        2.01         20.36  25.17  50.65    NaN   
4       30566.0  35100.0        0.45         20.22  19.31  43.43    NaN   
28      17616.0  39192.0        1.71         22.13  40.31  71.36  109.0   
6        1737.0  44047.0        2.44          1.15  28.77  30.25   77.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
57     0.03        1.20    16.71     XY24      NTT      POWER  
61     0.67        1.42    28.75     X40N      ATH     CABLES  
4      0.87        1.53    26.00     XY24      NTT       MISC  
28     0.45        0.82    32.79     XY24      BTT  INSURANCE  
6      0.04        1.29    21.18     X40N      NTT    FINANCE

In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction     RSP   Current  \
18       CAMS  3643.00  5250.99    -8.32       H-SC   24.74  101388.0   
96      WIPRO   243.46   420.00   -12.43       M-LC   14.43  150238.0   
12  BANKINDIA   113.49   190.00   -28.40       M-MC   37.63  125510.0   
94     VOLTAS  1272.00  1929.20   -20.67       H-MC    4.64  135058.0   
1    AARTIIND   487.04   919.00    18.42       M-SC  100.00   74933.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%   XIRR  \
18       -616.0   45635.0        0.36         -0.60   45.01  44.14  261.0   
96       -707.0  110155.0        0.47         -0.47   73.32  72.51   33.0   
12      -3642.0   90706.0        1.12         -2.82   72.27  67.42  -12.0   
94      -4862.0   77159.0       -0.15         -3.47   57.13  51.67    NaN   
1       -6403.0   78537.0        3.08         -7.87  104.81  88.69   -8.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
18    -0.01        0.85    18.73       XR      ATH       MISC  
96    -0.01        1.27    13.12       XR      NTT         IT  
12    -0.04        1.06    22.07       XR      NTT      BANKS  
94    -0.06        1.14     2.01       AR      ATH         AC  
1     -0.08        0.63    26.36       XR      NTT  CHEMICALS

In [14]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
35    HDFCLIFE   562.05   897.00    -4.90       H-LC  68.04   90669.0   
34    HDFCBANK  1561.04  2170.00     3.67       X-LC  72.16  102433.0   
89  UJJIVANSFB    52.77    60.00   -42.86       M-SC  84.54  111132.0   
73     SBILIFE  1496.49  1924.99    -1.70       H-LC  89.69  146126.0   
14  BERGEPAINT   558.54   680.00   -18.97       H-MC  52.58  173795.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  XIRR  \
35      20413.0  21452.0        0.38         29.05  23.66  59.59  16.0   
34      19698.0  12579.0        0.05         23.81  12.28  39.01   4.0   
89     -31347.0  50865.0        0.46        -22.00  45.77  13.70 -27.0   
73      20421.0  15577.0        0.79         16.25  10.66  28.63   NaN   
14      -3262.0  41763.0        0.92         -1.84  24.03  21.75 -14.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
35     0.95        0.76    36.18      X40      BTT  INSURANCE  
34     1.57        0.86    35.34      X40      BTT      BANKS  
89    -0.62        0.94    31.17     X40N      NTT      BANKS  
73     1.31        1.23    30.18       AR      ATH  INSURANCE  
14    -0.08        1.46    24.32     XY24      NTT     PAINTS

In [15]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

Symbol    AvgCost        FTT  Dev%_PE Conviction     RSP   Current  \
1   AARTIIND     487.04     919.00    18.42       M-SC  100.00   74933.0   
79  SONACOMS     578.05    1006.00   -31.35       M-MC   98.97   89215.0   
61   POLYCAB    5005.92    7541.35    18.73       H-MC   97.94  168700.0   
55       MRF  113820.50  150504.12    16.95       X-MC   96.91  268820.0   
66  RELIANCE    1288.53    1526.00     4.05       X-LC   95.88  223602.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  XIRR  \
1       -6403.0  78537.0        3.08         -7.87  104.81  88.69  -8.0   
79     -11944.0  86833.0        1.46        -11.81   97.33  74.03  -8.0   
61      28534.0  42462.0        2.01         20.36   25.17  50.65   NaN   
55      41179.0  32178.0       -0.43         18.09   11.97  32.23   NaN   
66      20014.0  17508.0       -0.40          9.83    7.83  18.43  67.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
1     -0.08        0.63    26.36       XR      NTT   CHEMICALS  
79    -0.14        0.75    25.38       AR      BTT        AUTO  
61     0.67        1.42    28.75     X40N      ATH      CABLES  
55     1.28        2.26    30.93       AR      ATH       TYRES  
66     1.14        1.88    21.80     XY25      NTT  REFINERIES

In [16]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

Top_N  Sum_Alloc%
0     10       18.29
1     25       40.30
2     50       67.87

In [17]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    36.07
X40     15.15
X40N    13.78
AR      12.97
XR      10.29
XY25     9.40
X200     1.16
SR       1.15
Name: CurrAlloc%, dtype: float64

In [18]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.92
X-LC    17.62
M-SC    16.01
X-MC     8.05
M-LC     7.98
M-MC     7.58
H-MC     7.20
H-LC     6.28
X-SC     2.56
L-SC     2.24
Name: CurrAlloc%, dtype: float64

In [19]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

CurrAlloc%  Current P/L%    FTT%
Category                                       
BANKS               11.98        -11.79   66.03
FMCG                11.06         -2.69   34.12
IT                   8.94        -28.34   95.52
FINANCE              7.29        -27.90   85.61
INSURANCE            7.03         -1.81   38.94
MISC                 6.34        -36.72   86.19
PAINTS               5.77        -20.04   55.56
ELECTRICAL           4.21        -11.38   68.13
HEALTHCARE           3.67         -6.72   41.17
CHEMICALS            3.26        -21.88  105.85
AUTO                 2.81        -23.20   77.37
FOOTWEAR             2.44        -42.49   89.57
TYRES                2.26         15.32   11.97
POWER                2.07        -15.02   61.03
TRAVEL               1.96        -36.67   95.81
AC                   1.94        -16.20   78.20
REFINERIES           1.88          8.95    7.83
TEXTILES             1.66        -16.74   78.06
DURABLES             1.64         -2.82   81.59
STEEL                1.63        -16.37  103.10
CERAMICS             1.43        -67.54  135.70
CABLES               1.42         16.91   25.17
JEWELLERY            1.20       -102.22  166.17
CEMENT               1.13        -40.36  112.58
ENGINEERING          1.03        -57.41  122.08
ENTERTAINMENT        0.87        -57.66  136.93
PHARMA               0.86          0.44   49.14
REALTY               0.85         -1.25   52.54
MINING               0.84         -0.46   25.54
HOTELS               0.50        -92.39  160.25

In [20]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Criteria                  
XY24      3279585.0     29
XR        1322172.0     15
X40N      1265156.0     17
AR        1104240.0     13
X40        799424.0     14
SR         208025.0      2
XY25       178408.0      6
X200        80654.0      1

In [21]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction                  
H-SC        2366861.0     21
M-SC        2017886.0     20
X-LC         763056.0     13
M-MC         690774.0      7
H-MC         403220.0      7
X-SC         373287.0      4
M-LC         369018.0      6
X-MC         350121.0      6
L-SC         331785.0      4
H-LC         247848.0      6

In [22]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      977290.0      6
M-SC       XY24      823045.0      6
H-SC       X40N      592210.0      7
M-MC       XY24      487710.0      4
M-SC       XR        452199.0      5
X-LC       X40       404156.0      6
M-SC       X40N      396722.0      5
H-SC       AR        392818.0      2
           XR        314458.0      4
M-LC       XY24      235313.0      4
H-MC       XY24      194871.0      3
X-LC       AR        193357.0      3
L-SC       XR        166539.0      2
M-SC       X40       166282.0      2
H-LC       AR        157310.0      3
X-MC       XY24      142284.0      2
X-SC       XR        132367.0      1
           SR        122990.0      1
           X40N      117930.0      2
X-MC       X40N      115832.0      2
M-LC       XR        110155.0      1
L-SC       XY24       95264.0      1
M-SC       AR         94603.0      1
M-MC       XR         90706.0      1
H-SC       X40        90085.0      2
M-MC       AR         86833.0      1
M-SC       SR         85035.0      1
X-LC       XY25       84889.0      3
           X200       80654.0      1
H-MC       AR         77159.0      1
L-SC       AR         69982.0      1
X-MC       X40        59827.0      1
H-MC       XR         55748.0      1
H-LC       X40        46094.0      2
           XY25       44444.0      1
H-MC       X40N       42462.0      1
           X40        32980.0      1
X-MC       AR         32178.0      1
M-MC       XY25       25525.0      1
M-LC       XY25       23550.0      1

In [23]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 40.0 seconds
